In [ ]:
import polars as pl

In [ ]:
# csv 파일 경로 지정
csv_file_path = "./data/deoksan.csv"  # 실제 CSV 파일 경로로 변경하세요

# CSV 파일 읽기 (Polars 사용)
df = pl.read_csv(csv_file_path)

In [ ]:
df.columns

['time',
 'curr',
 'currR',
 'currS',
 'currT',
 'Ground',
 'PT100',
 'Vibra',
 'Volt',
 'VoltR',
 'VoltS',
 'VoltT']

In [ ]:
df[:2]

time,curr,currR,currS,currT,Ground,PT100,Vibra,Volt,VoltR,VoltS,VoltT
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2021-04-01T00:00:00.200""",200.0,222.1,224.4,225.8,222.3,223.3,223.0,219.4,221.0,218.4,221.9
"""2021-04-01T00:00:00.300""",300.0,222.0,224.4,225.8,222.3,223.3,223.0,219.4,220.9,218.4,221.9


# 업데이트

In [ ]:
from owlready2 import *
from datetime import datetime
# 매핑 온톨로지 객체 생성 (URL 지정)
onto = get_ontology("http://pcnrnd.org/mapping.owl")

with onto:
    # 산업 클래스   
    class Industry(Thing): pass

    # 기계 클래스
    class Machine(Industry): pass
    
    # 산업 데이터 클래스
    class Data(Machine): pass

    # 머신러닝 모델 클래스
    class MLModel(Machine): pass

    # 머신러닝 데이터 클래스
    class MLMetricData(Data): pass

    # 데이터 속성
    class has_machine(ObjectProperty):
        domain = [Industry]
        range = [Machine]

    class has_data(ObjectProperty):
        domain = [Machine]
        range = [Data]
    
    class has_ml_model(ObjectProperty):
        domain = [Data]
        range = [MLModel]

    class has_ml_metric_data(ObjectProperty):
        domain = [MLModel]
        range = [MLMetricData]
    
    class ml_metric_similar_to(ObjectProperty):
        domain = [MLMetricData]
        range = [MLMetricData]

    class very_similar_to(ObjectProperty):
        """매우 유사한 데이터 (0.95 이상)"""
        domain = [MLMetricData]
        range = [MLMetricData]
    
    class similar_to(ObjectProperty):
        """유사한 데이터 (0.9 이상)"""
        domain = [MLMetricData]
        range = [MLMetricData]
    
    class moderately_similar_to(ObjectProperty):
        """보통 유사한 데이터 (0.8 이상)"""
        domain = [MLMetricData]
        range = [MLMetricData]


    class matches_with(ObjectProperty):
        domain = [MLMetricData]
        range = [MLMetricData]

    class timestamp(DataProperty, FunctionalProperty):
        domain = [Data]
        range = [datetime]

    class float_value(DataProperty):
        domain = [Data]
        range = [float]

    # MLMetricData의 메트릭 속성들
    class accuracy(DataProperty, FunctionalProperty):
        domain = [MLMetricData]
        range = [float]
    
    class precision(DataProperty, FunctionalProperty):
        domain = [MLMetricData]
        range = [float]
    
    class recall(DataProperty, FunctionalProperty):
        domain = [MLMetricData]
        range = [float]
    
    class f1_score(DataProperty, FunctionalProperty):
        domain = [MLMetricData]
        range = [float]

    class similarity_score(DataProperty, FunctionalProperty):
        domain = [MLMetricData]
        range = [float]
    
    # 추론 규칙
    class MonitoredMachine(Machine):
        equivalent_to = [Machine & has_data.some(Data)]

    class MonitoredData(Data):
        equivalent_to = [Data & has_ml_model.some(MLModel)]

    class EvaluatedMLModel(MLModel):
        equivalent_to = [MLModel & has_ml_metric_data.some(MLMetricData)]
        # 유사한 MLMetricData를 자동으로 분류하는 규칙

    class SimilarMLMetricData(MLMetricData):
        equivalent_to = [MLMetricData & ml_metric_similar_to.some(MLMetricData)]

    class SimilarMLMetricData(MLMetricData):
        """유사한 데이터 (기본 - 모든 단계 포함)"""
        equivalent_to = [MLMetricData & ml_metric_similar_to.some(MLMetricData)]
    
    class VerySimilarMLMetricData(MLMetricData):
        """매우 유사한 데이터 (0.95 이상)"""
        equivalent_to = [MLMetricData & very_similar_to.some(MLMetricData)]
    
    class HighSimilarMLMetricData(MLMetricData):
        """높은 유사도 데이터 (0.9 이상)"""
        equivalent_to = [MLMetricData & similar_to.some(MLMetricData)]
    
    class ModerateSimilarMLMetricData(MLMetricData):
        """보통 유사도 데이터 (0.8 이상)"""
        equivalent_to = [MLMetricData & moderately_similar_to.some(MLMetricData)]
    

In [ ]:
onto.save("mapping.owl")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_similarity_metric_tiered(metric1, metric2):
    """
    단계별 유사도 매칭 함수 (포함 관계, 중복 방지)
    - 0.95 이상: very_similar_to + similar_to + moderately_similar_to 모두
    - 0.9 이상: similar_to + moderately_similar_to
    - 0.8 이상: moderately_similar_to만
    - 0.8 미만: 매칭 안 함
    """
    # 🔥 중복 체크: 이미 관계가 설정되어 있으면 스킵
    if metric2 in metric1.ml_metric_similar_to:
        print(f"⚠️ 이미 매칭된 데이터입니다. (유사도: {metric1.similarity_score})")
        return "already_matched"
    
    vec1 = np.array([
        metric1.accuracy, 
        metric1.precision, 
        metric1.recall, 
        metric1.f1_score
    ]).reshape(1, -1)
    
    vec2 = np.array([
        metric2.accuracy, 
        metric2.precision, 
        metric2.recall, 
        metric2.f1_score
    ]).reshape(1, -1)
    
    similarity = float(cosine_similarity(vec1, vec2)[0][0])
    
    # 기본 관계는 항상 설정
    metric1.ml_metric_similar_to.append(metric2)
    metric2.ml_metric_similar_to.append(metric1)
    metric1.similarity_score = similarity
    metric2.similarity_score = similarity
    
    # 단계별 threshold 적용 (포함 관계)
    if similarity >= 0.95:
        # 매우 유사: 모든 단계에 추가 (포함 관계)
        metric1.very_similar_to.append(metric2)
        metric2.very_similar_to.append(metric1)
        metric1.similar_to.append(metric2)
        metric2.similar_to.append(metric1)
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        print(f"🟢 매우 유사 ({similarity:.4f}) - 모든 단계에 추가")
        return "very_similar"
        
    elif similarity >= 0.9:
        # 유사: similar_to와 moderately_similar_to에 추가
        metric1.similar_to.append(metric2)
        metric2.similar_to.append(metric1)
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        print(f"🟡 유사 ({similarity:.4f}) - similar_to와 moderately_similar_to에 추가")
        return "similar"
        
    elif similarity >= 0.8:
        # 보통: moderately_similar_to만 추가
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        print(f"🟠 보통 유사 ({similarity:.4f}) - moderately_similar_to만 추가")
        return "moderate"
        
    else:
        # 낮음: 매칭 안 함
        print(f"🔴 낮은 유사도 ({similarity:.4f}) - 매칭 안 함")
        return "low"

In [ ]:
def cosine_similarity_metric_tiered(metric1, metric2):
    """
    단계별 유사도 매칭 함수 (포함 관계, 기본 관계 유지)
    """
    # 중복 체크
    if metric2 in metric1.ml_metric_similar_to:
        print(f"⚠️ 이미 매칭된 데이터입니다.")
        return "already_matched"
    
    vec1 = np.array([
        metric1.accuracy, metric1.precision, 
        metric1.recall, metric1.f1_score
    ]).reshape(1, -1)
    
    vec2 = np.array([
        metric2.accuracy, metric2.precision, 
        metric2.recall, metric2.f1_score
    ]).reshape(1, -1)
    
    similarity = float(cosine_similarity(vec1, vec2)[0][0])
    
    # ✅ 기본 관계는 항상 설정 (추론 규칙과 쿼리를 위해)
    metric1.ml_metric_similar_to.append(metric2)
    metric2.ml_metric_similar_to.append(metric1)
    metric1.similarity_score = similarity
    metric2.similarity_score = similarity
    
    # 단계별 threshold 적용 (포함 관계)
    if similarity >= 0.95:
        metric1.very_similar_to.append(metric2)
        metric2.very_similar_to.append(metric1)
        metric1.similar_to.append(metric2)
        metric2.similar_to.append(metric1)
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        return "very_similar"
    elif similarity >= 0.9:
        metric1.similar_to.append(metric2)
        metric2.similar_to.append(metric1)
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        return "similar"
    elif similarity >= 0.8:
        metric1.moderately_similar_to.append(metric2)
        metric2.moderately_similar_to.append(metric1)
        return "moderate"
    else:
        # 0.8 미만이면 기본 관계도 제거 (선택사항)
        metric1.ml_metric_similar_to.remove(metric2)
        metric2.ml_metric_similar_to.remove(metric1)
        return "low"

In [ ]:
# 두 개의 MLMetricData 생성
ml_metric1 = MLMetricData("model1_metrics")
ml_metric1.accuracy = 0.95
ml_metric1.precision = 0.93
ml_metric1.recall = 0.94
ml_metric1.f1_score = 0.935

ml_metric2 = MLMetricData("model2_metrics")
ml_metric2.accuracy = 0.94
ml_metric2.precision = 0.92
ml_metric2.recall = 0.95
ml_metric2.f1_score = 0.93

# 단계별 유사도 매칭 실행
result = cosine_similarity_metric_tiered(ml_metric1, ml_metric2)
print(f"매칭 결과: {result}")

# 관계 확인
print(f"\n매우 유사한 데이터:")
print(f"  ml_metric1: {ml_metric1.very_similar_to}")
print(f"  ml_metric2: {ml_metric2.very_similar_to}")

print(f"\n유사한 데이터:")
print(f"  ml_metric1: {ml_metric1.similar_to}")
print(f"  ml_metric2: {ml_metric2.similar_to}")

print(f"\n보통 유사한 데이터:")
print(f"  ml_metric1: {ml_metric1.moderately_similar_to}")
print(f"  ml_metric2: {ml_metric2.moderately_similar_to}")

# 추론기 실행 후 자동 분류 확인
sync_reasoner()
print(f"\nml_metric1 분류: {ml_metric1.is_a}")
print(f"ml_metric2 분류: {ml_metric2.is_a}")

⚠️ 이미 매칭된 데이터입니다. (유사도: 0.999961570227192)
매칭 결과: already_matched

매우 유사한 데이터:
  ml_metric1: [mapping.model2_metrics, mapping.model2_metrics, mapping.model2_metrics, mapping.model2_metrics, mapping.model2_metrics]
  ml_metric2: [mapping.model1_metrics, mapping.model1_metrics, mapping.model1_metrics, mapping.model1_metrics, mapping.model1_metrics]

유사한 데이터:
  ml_metric1: [mapping.model2_metrics, mapping.model2_metrics]
  ml_metric2: [mapping.model1_metrics, mapping.model1_metrics]

보통 유사한 데이터:
  ml_metric1: [mapping.model2_metrics, mapping.model2_metrics]
  ml_metric2: [mapping.model1_metrics, mapping.model1_metrics]


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp f:\project_test\industry\ontology\.venv\lib\site-packages\owlready2\hermit;f:\project_test\industry\ontology\.venv\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/PCN/AppData/Local/Temp/tmp3ju4isp6



ml_metric1 분류: [mapping.SimilarMLMetricData, mapping.VerySimilarMLMetricData, mapping.ModerateSimilarMLMetricData, mapping.HighSimilarMLMetricData]
ml_metric2 분류: [mapping.SimilarMLMetricData, mapping.VerySimilarMLMetricData, mapping.ModerateSimilarMLMetricData, mapping.HighSimilarMLMetricData]


* Owlready2 * HermiT took 0.8736696243286133 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [31]:
# 두 개의 MLMetricData 생성
ml_metric1 = MLMetricData("model1_metrics")
ml_metric1.accuracy = 0.95      # [0.95] → 0.95로 변경
ml_metric1.precision = 0.93     # [0.93] → 0.93로 변경
ml_metric1.recall = 0.94       # [0.94] → 0.94로 변경
ml_metric1.f1_score = 0.935    # [0.935] → 0.935로 변경

ml_metric2 = MLMetricData("model2_metrics")
ml_metric2.accuracy = 0.94     # [0.94] → 0.94로 변경
ml_metric2.precision = 0.92    # [0.92] → 0.92로 변경
ml_metric2.recall = 0.95       # [0.95] → 0.95로 변경
ml_metric2.f1_score = 0.93     # [0.93] → 0.93로 변경

In [ ]:
industry = Industry("manufacturing")
machine = Machine("molten_aluminum_machine")
data = Data("molten_aluminum_machine_data")
ml_model = MLModel("molten_aluminum_machine_ml_model")
ml_metric_data = MLMetricData("molten_aluminum_machine_ml_metric_data")


industry.has_machine = [machine]
machine.has_data = [data]
data.has_ml_model = [ml_model]
ml_model.has_ml_metric_data = [ml_metric_data]

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_similarity_metric(metric1, metric2, threshold=0.9):
    """코사인 유사도 (방향성 고려, -1~1, 1에 가까울수록 유사)"""
    vec1 = np.array([
        metric1.accuracy, 
        metric1.precision, 
        metric1.recall, 
        metric1.f1_score
    ]).reshape(1, -1)
    
    vec2 = np.array([
        metric2.accuracy, 
        metric2.precision, 
        metric2.recall, 
        metric2.f1_score
    ]).reshape(1, -1)
    
    similarity = cosine_similarity(vec1, vec2)[0][0]
    
    # numpy.float64를 Python float로 변환
    similarity_float = float(similarity)
    
    # 🔥 추가: 유사도가 임계값 이상이면 관계 설정
    if similarity_float >= threshold:
        metric1.ml_metric_similar_to.append(metric2)
        metric2.ml_metric_similar_to.append(metric1)
        metric1.similarity_score = similarity_float  # float로 변환
        metric2.similarity_score = similarity_float   # float로 변환
        print(f"✅ 유사도 {similarity_float:.4f}로 매칭 완료!")
    else:
        print(f"❌ 유사도 {similarity_float:.4f}는 임계값 {threshold} 미만")
    
    return similarity_float  # float로 반환

In [28]:
# 유사성 매칭 실행 (임계값 명시)
similarity = cosine_similarity_metric(ml_metric1, ml_metric2)
print(f"계산된 유사도: {similarity:.4f}")

# 관계 확인
print(f"ml_metric1과 유사한 데이터: {ml_metric1.ml_metric_similar_to}")
print(f"ml_metric2와 유사한 데이터: {ml_metric2.ml_metric_similar_to}")

# 추론기 실행 후 자동 분류 확인
sync_reasoner()
print(f"ml_metric1 분류: {ml_metric1.is_a}")  # SimilarMLMetricData 포함될 수 있음
print(f"ml_metric2 분류: {ml_metric2.is_a}")

✅ 유사도 1.0000로 매칭 완료!
계산된 유사도: 1.0000
ml_metric1과 유사한 데이터: [mapping.model2_metrics, mapping.model2_metrics, mapping.model2_metrics, mapping.model2_metrics]
ml_metric2와 유사한 데이터: [mapping.model1_metrics, mapping.model1_metrics, mapping.model1_metrics, mapping.model1_metrics]


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp f:\project_test\industry\ontology\.venv\lib\site-packages\owlready2\hermit;f:\project_test\industry\ontology\.venv\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/PCN/AppData/Local/Temp/tmp9w7s05ic


ml_metric1 분류: [mapping.SimilarMLMetricData]
ml_metric2 분류: [mapping.SimilarMLMetricData]


* Owlready2 * HermiT took 0.864457368850708 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [6]:
ml_model

mapping.molten_aluminum_machine_ml_model

In [ ]:
df.columns

['time',
 'curr',
 'currR',
 'currS',
 'currT',
 'Ground',
 'PT100',
 'Vibra',
 'Volt',
 'VoltR',
 'VoltS',
 'VoltT']

In [4]:
print(industry.has_machine)
print(machine.has_data)
print(data.has_ml_model)
print(ml_model.has_ml_metric_data)

[mapping.molten_aluminum_machine]
[mapping.molten_aluminum_machine_data]
[mapping.molten_aluminum_machine_ml_model]
[mapping.molten_aluminum_machine_ml_metric_data]


In [ ]:
# onto.save("mapping.owl")

In [9]:
print(onto.has_machine.get_relations())
print(onto.has_data.get_relations())
print(onto.has_ml_model.get_relations())
print(onto.has_ml_metric_data.get_relations())

<generator object Property.get_relations at 0x0000016515427840>
<generator object Property.get_relations at 0x0000016515427840>
<generator object Property.get_relations at 0x0000016515427840>
<generator object Property.get_relations at 0x0000016515427840>
